In [1]:
from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import BayesianNetwork

# Establishing the structure
model = BayesianNetwork([
    ('Sensors', 'Data'),
    ('Sensors', 'Battery'),
    ('Data', 'Safety'),
    ('Data', 'Quality'),
    ('Quality', 'Precision'),
    ('Safety', 'Precision'),
    ('Precision', 'Utility'),
])

# Defining the relationships
Sensors_cpd = TabularCPD(
    variable='Sensors',
    variable_card=2,
    values=[[.75], [.25]]
)

Data_cpd = TabularCPD(
    variable='Data',
    variable_card=2,
    values=[[.8, .9], [.2, .1]],
    evidence=['Sensors'],
    evidence_card=[2]
)

Battery_cpd = TabularCPD(
    variable='Battery',
    variable_card=2,
    values=[[.8, .9], [.2, .1]],
    evidence=['Sensors'],
    evidence_card=[2]
)

Safety_cpd = TabularCPD(
    variable='Safety',
    variable_card=2,
    values=[[.6, .3], [.4, .7]],
    evidence=['Data'],
    evidence_card=[2]
)

Quality_cpd = TabularCPD(
    variable='Quality',
    variable_card=2,
    values=[[.6, .3], [.4, .7]],
    evidence=['Data'],
    evidence_card=[2]
)

Precision_cpd = TabularCPD(
    variable='Precision',
    variable_card=2,
    values=[[.9, .4, .2, .3], [.1, .6, .8, .7]],
    evidence=['Safety', 'Quality'],
    evidence_card=[2, 2]
)

Utility_cpd = TabularCPD(
    variable='Utility',
    variable_card=2,
    values=[[.6, .3], [.4, .7]],
    evidence=['Precision'],
    evidence_card=[2]
)

# Adding the relationships to the model
model.add_cpds(Sensors_cpd, Data_cpd, Battery_cpd, Safety_cpd, Quality_cpd, Precision_cpd, Utility_cpd)

# Verifying the structure of the model
model.get_cpds()

[<TabularCPD representing P(Sensors:2) at 0x15da968a950>,
 <TabularCPD representing P(Data:2 | Sensors:2) at 0x15da9678d10>,
 <TabularCPD representing P(Battery:2 | Sensors:2) at 0x15da9667010>,
 <TabularCPD representing P(Safety:2 | Data:2) at 0x15da9667150>,
 <TabularCPD representing P(Quality:2 | Data:2) at 0x15da9666a10>,
 <TabularCPD representing P(Precision:2 | Safety:2, Quality:2) at 0x15da96ef290>,
 <TabularCPD representing P(Utility:2 | Precision:2) at 0x15da96ef490>]

In [2]:
# Find the node paths to determine what provides information about a node
print(model.active_trail_nodes('Precision'))
print()

print(model.active_trail_nodes('Safety'))
print()

print(model.active_trail_nodes('Data'))

{'Precision': {'Utility', 'Quality', 'Safety', 'Data', 'Battery', 'Sensors', 'Precision'}}

{'Safety': {'Utility', 'Quality', 'Safety', 'Data', 'Battery', 'Sensors', 'Precision'}}

{'Data': {'Utility', 'Quality', 'Safety', 'Data', 'Battery', 'Sensors', 'Precision'}}


In [3]:
# Determination of local independencies
print(model.local_independencies('Sensors'))
print()

print(model.local_independencies('Quality'))
print()

print(model.local_independencies('Safety'))
print()

model.get_independencies()




(Quality ⟂ Battery, Sensors, Safety | Data)

(Safety ⟂ Battery, Sensors, Quality | Data)



(Utility ⟂ Battery, Sensors | Data)
(Utility ⟂ Battery | Sensors)
(Utility ⟂ Quality, Safety, Data, Battery, Sensors | Precision)
(Utility ⟂ Battery, Sensors, Data | Quality, Safety)
(Utility ⟂ Battery, Sensors | Quality, Data)
(Utility ⟂ Battery | Sensors, Quality)
(Utility ⟂ Battery, Sensors, Safety, Data | Quality, Precision)
(Utility ⟂ Battery, Sensors | Safety, Data)
(Utility ⟂ Battery | Sensors, Safety)
(Utility ⟂ Battery, Sensors, Quality, Data | Safety, Precision)
(Utility ⟂ Sensors | Battery, Data)
(Utility ⟂ Battery | Sensors, Data)
(Utility ⟂ Safety, Battery, Sensors, Quality | Precision, Data)
(Utility ⟂ Safety, Sensors, Quality, Data | Battery, Precision)
(Utility ⟂ Battery, Quality, Safety, Data | Sensors, Precision)
(Utility ⟂ Battery, Sensors | Quality, Safety, Data)
(Utility ⟂ Sensors, Data | Battery, Quality, Safety)
(Utility ⟂ Battery, Data | Sensors, Quality, Safety)
(Utility ⟂ Battery, Sensors, Data | Quality, Safety, Precision)
(Utility ⟂ Sensors | Battery, Qualit

In [5]:
from pgmpy.inference import VariableElimination
# Calculate existing probabilities to verify everything is correct
inference = VariableElimination(model)
prob_sensors = inference.query(variables=['Sensors'])
print(prob_sensors)

+------------+----------------+
| Sensors    |   phi(Sensors) |
+============+================+
| Sensors(0) |         0.7500 |
+------------+----------------+
| Sensors(1) |         0.2500 |
+------------+----------------+


In [8]:
# Perform another probability calculation
prob_data = inference.query(variables=['Data'])
print(prob_data)

+---------+-------------+
| Data    |   phi(Data) |
+=========+=============+
| Data(0) |      0.8250 |
+---------+-------------+
| Data(1) |      0.1750 |
+---------+-------------+


In [9]:
# conditional probability
# The probability of achieving good data precision when data safety is ensured 
proba_precision_safety = inference.query(variables=['Precision'], evidence={'Safety': 1}) 
print(proba_precision_safety)

+--------------+------------------+
| Precision    |   phi(Precision) |
+==============+==================+
| Precision(0) |           0.2481 |
+--------------+------------------+
| Precision(1) |           0.7519 |
+--------------+------------------+


In [10]:
# Probability of achieving good precision when safety is lacking but quality is good
proba_precision_safety_quality = inference.query(variables=['Precision'], evidence={'Safety': 0, 'Quality': 1})
print(proba_precision_safety_quality)

+--------------+------------------+
| Precision    |   phi(Precision) |
+==============+==================+
| Precision(0) |           0.4000 |
+--------------+------------------+
| Precision(1) |           0.6000 |
+--------------+------------------+
